In [1]:
# Update reaction annotations
import collections
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [2]:
one_biomd = 'BIOMD0000000190.xml'
one_biomd_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
reader = libsbml.SBMLReader()
document = reader.readSBML(one_biomd_fpath)
model = document.getModel()

In [3]:
model.getListOfSpecies()

<ListOfSpecies[11]>

In [4]:
from AMAS import recommender
recom = recommender.Recommender(libsbml_fpath=one_biomd_fpath)

In [5]:
# Next step..? add getRecall and getPrecision
recom = recommender.Recommender(libsbml_fpath=ecoli_fpath)

In [6]:
# recom.reactions.reaction_components

In [11]:
recom.getReactionAnnotation('R_PFK')

TypeError: unhashable type: 'list'

In [9]:
recom.reactions.reaction_components

{'R_PFK': ['M_fdp_c', 'M_f6p_c', 'M_h_c', 'M_atp_c', 'M_adp_c'],
 'R_PFL': ['M_pyr_c', 'M_coa_c', 'M_for_c', 'M_accoa_c'],
 'R_PGI': ['M_f6p_c', 'M_g6p_c'],
 'R_PGK': ['M_13dpg_c', 'M_3pg_c', 'M_adp_c', 'M_atp_c'],
 'R_PGL': ['M_6pgc_c', 'M_h_c', 'M_h2o_c', 'M_6pgl_c'],
 'R_ACALD': ['M_accoa_c',
  'M_acald_c',
  'M_nad_c',
  'M_h_c',
  'M_nadh_c',
  'M_coa_c'],
 'R_AKGt2r': ['M_akg_e', 'M_h_c', 'M_akg_c', 'M_h_e'],
 'R_PGM': ['M_2pg_c', 'M_3pg_c'],
 'R_PIt2r': ['M_pi_e', 'M_h_c', 'M_pi_c', 'M_h_e'],
 'R_ALCD2x': ['M_etoh_c', 'M_acald_c', 'M_nad_c', 'M_h_c', 'M_nadh_c'],
 'R_ACALDt': ['M_acald_c', 'M_acald_e'],
 'R_ACKr': ['M_ac_c', 'M_actp_c', 'M_adp_c', 'M_atp_c'],
 'R_PPC': ['M_pi_c', 'M_h_c', 'M_h2o_c', 'M_pep_c', 'M_co2_c', 'M_oaa_c'],
 'R_ACONTa': ['M_cit_c', 'M_h2o_c', 'M_acon_C_c'],
 'R_ACONTb': ['M_icit_c', 'M_h2o_c', 'M_acon_C_c'],
 'R_ATPM': ['M_pi_c', 'M_h_c', 'M_h2o_c', 'M_atp_c', 'M_adp_c'],
 'R_PPCK': ['M_atp_c', 'M_pep_c', 'M_co2_c', 'M_adp_c', 'M_oaa_c'],
 'R_ACt2r': ['

# Updating test_reaction_annotation

In [16]:
R_PFK = 'R_PFK'
ATP = 'M_atp_c'
COMPONENTS = {'M_fdp_c', 'M_adp_c', 'M_atp_c', 'M_f6p_c', 'M_h_c'}
ONE_CANDIDATE = 'RHEA:12423'
ONE_CHEBI = 'CHEBI:30616'
BIOMD_248_PATH = os.path.join(os.getcwd(), 'BIOMD0000000248.xml')
E_COLI_PATH = os.path.join(BIGG_DIR, "e_coli_core.xml")

from AMAS import recommender
recom = recommender.Recommender(libsbml_fpath=ecoli_fpath)

In [19]:
spec_cl = sa.SpeciesAnnotation(libsbml_fpath = E_COLI_PATH)
reac_cl = ra.ReactionAnnotation(libsbml_fpath = E_COLI_PATH)
pred_species = spec_cl.predictAnnotationByName(inp_spec_list=COMPONENTS)
pred_reaction = reac_cl.predictAnnotation(inp_spec_dict=spec_cl.formula,
                                          inp_reac_list=[R_PFK])

In [23]:
print([val[0] for val in reac_cl.match_score[R_PFK]])
ONE_CANDIDATE in [val[0] for val in reac_cl.match_score[R_PFK]]

['RHEA:12423', 'RHEA:13380', 'RHEA:14216', 'RHEA:15656', 'RHEA:16112', 'RHEA:20108']


True

In [31]:
one_match_score = {'R1': [('RHEA:1', 1.0), ('RHEA:2', 0.5)]}
reac_cl.getBestOneCandidates(one_match_score)['R1']

['RHEA:1']

In [32]:
reac_cl.candidates

{'R_PFK': Index(['RHEA:12423', 'RHEA:13380', 'RHEA:14216', 'RHEA:15656', 'RHEA:16112',
        'RHEA:20108'],
       dtype='object')}

In [33]:
pred = {R_PFK: ['RHEA:16112']}
reac_cl.getAccuracy(pred_annotation=pred)

1.0